In [6]:
import sys
import numpy as np
import pybullet as p
import math
import time
import pybullet_data
import controlpy

In [7]:
# # LQR Controller

# Here, we are working on LQR controller. We have used control library [[1](#PythonControl)] of python to work on lqr. 
# For the lqr controllers, the state-space equations of the system are required. 
# The above equation is derived from [[2](#SystemEquation)] .
# Here,
#(M)       mass of the cart                         2 kg
#(m)       mass of the pendulum                     5 kg
#(b)       coefficient of friction for cart         0.5 N/m/sec
#(l)       length to pendulum center of mass        0.1 m
#(I)       mass moment of inertia of the pendulum   0.10500391309238813 kg.m^2
#(F)       force applied to the cart
#(x)       cart position coordinate
#(theta)   pendulum angle from vertical (down)

#We can calculate
#x(t+1)=A*x(t)+B*u
#x(t+1)=k*x(t)

M = 2;
m = 5;
b = 0.5;
I = 0.10500391309238813;
g = 9.8;
l = 0.1;

px = I*(M+m)+pow(M*m*l,2) #denominator for the A and B matrices

A = np.array([[0,1],[m*g*l*(M+m)/px,0]])#considering theta and theta dot only 
B = np.array([[  0.  ],
       [ m*l/px]])

In [8]:
# The class for LQR

# Q = np.array([[ 100,   0],[  0, 1000]])
# R = 0.0001
# K,S,e = lqr(A,B,Q,R)
# print(K)
# print(S)
# print(e)

# From [[2](#SystemEquation)] , we get the cost function 
# J = int (x^T Q x +u^T R u)dt
# Here, we need to tune Q matrix and R such that the value of "J" is minimum. The lqp function automatically does that for you

class SelfBalanceLQR:
    def __init__(self):
        self.xvelMin=-.01# x velocity
        self.xvelMax =0
        self.yMin = -0.01#yaw
        self.yMax = 0
        self.y_ = 0
        self.Q = np.array([[ 10,   0],[  0, 1000]])
        self.R = [[0.0001]]
        self.K,self.S,self.e = controlpy.synthesis.controller_lqr(A,B,self.Q,self.R)
        
    def callback(self,data,key):
        
        #vel = data[0][1]
        y = data[1][1]#theata changes along 1
        yTheta=y
        #for going forward or back we need to balance the bot at an angle, trying to balance at 10 degress
        if (key=="down"):
            yTheta=y-10*np.pi/180
        elif (key == "up" ):
            yTheta=y+10*np.pi/180
        
        diff_yaw = y-self.y_
        np_x = np.array([[yTheta],[diff_yaw]])
        
        u_t=-np.matmul(self.K,np_x) 
        #print(np_x,self.K,u_t)
        #print(np_x.shape,self.K.shape,u_t.shape,A.shape,B.shape)
        xvel = (np.matmul(A,np_x)+np.matmul(B,u_t))[1]
        #print(xvel)
        
        if y>self.yMax:
            self.yMax = y
        elif y<self.yMin:
            self.yMin =y
        if xvel>self.xvelMax:
            self.xvelMax=xvel
        elif xvel<self.xvelMin:
            self.xvelMin = xvel
            
        linear_vel = [xvel,0,0]
        angular_vel=[0,0,0]
        #print("Max vel " + str(self.xvelMax) + " & Min vel " + str(self.xvelMin) + " Max y " + str(self.yMax*180/3.1416) +" & Min y" + str(self.yMin*180/3.1416))
        #print("Velocity "+ str(xvel)+ " & yaw " + str(y))
        self.y_ = y
        
        return xvel-np_x[1]
        
    def callback_q(self,data):
        q = data.data
        self.Q = np.array([[ q,   0],[  0, 10*q]])
        self.K,self.S,self.e = controlpy.synthesis.controller_lqr(A,B,self.Q,self.R)
        
    def callback_r(self,data):
        r = data.data
        self.R = r
        self.K,self.S,self.e = controlpy.synthesis.controller_lqr(A,B,self.Q,self.R)

In [9]:
def synthesizeData(p,robot):
    print("----------------------------------------------------------------------------------------------------------------")
    # print("Dynamic Info of Base : ",p.getDynamicsInfo(robot, -1),end="\n")
    # #0->mass , 3->local inertial pos
    # print("Base position and Orientation : " , p.getBasePositionAndOrientation(robot),end="\n")
    # #1->orientation

    # com = p.getDynamicsInfo(robot, -1)
    # com += p.getBasePositionAndOrientation(robot)[0][2] 
    # print("Centre of mass - ", com)

    #information required yaw
    #imu sensor , kp ,ki ,kd
    #set cmd_vel 
    position,orientation=p.getBasePositionAndOrientation(robot)
    euler_angles=np.array(p.getEulerFromQuaternion(orientation))#1->orientation
    deg_orien=euler_angles*180/np.pi
    #print(deg_orien)
    theta=deg_orien[1]
    #pos=position[0]
    velocity,angular=p.getBaseVelocity(robot)
    #print([velocity,euler_angles])
    data=[angular,euler_angles]
    return data

In [10]:
def getKeyPressed():
    x="no_button"
    keys = p.getKeyboardEvents()
    for k, v in keys.items():


        if (v & p.KEY_WAS_RELEASED):
            x="no_button"


        elif (v & p.KEY_IS_DOWN):

            if (k == p.B3G_LEFT_ARROW ):
                x="left"

                #print("left key is ressed")


            elif (k == p.B3G_RIGHT_ARROW ):
                x="right"

                #print("right key is ressed")


            elif (k == p.B3G_DOWN_ARROW ):
                x="down"

            #print("down key is pressed")


            if (k == p.B3G_UP_ARROW ):
                x="up"

                #print("up key is pressed")
    return x

In [ ]:
id = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
plane = p.loadURDF("plane.urdf")
p.setGravity(0, 0, -9.8)

robot = p.loadURDF("../urdf/self_balance.urdf" , [0,0,0.2])

# num = p.getNumJoints(robot)
# for i in range(num):
#     info = p.getJointInfo(robot, i)
#     print(info,end="\n")
#     link_name = info[12].decode("ascii")
#     if link_name == "left_wheel": left_wheel = j
#     if link_name == "right_wheel": wheel_foot = j
left_joint=0
right_joint=1
maxForce = 0
mode = p.VELOCITY_CONTROL
p.setJointMotorControl2(robot, left_joint,controlMode=mode, force=maxForce)
p.setJointMotorControl2(robot, right_joint,controlMode=mode, force=maxForce)



# p.changeDynamics(robot,left_joint,lateralFriction=0.7,spinningFriction=0.5,rollingFriction=0.5)
# p.changeDynamics(robot,right_joint,lateralFriction=0.7,spinningFriction=0.5,rollingFriction=0.5)

balance=SelfBalanceLQR()
count=0
while(True):
    
    left_tor=0
    right_tor=0
    key=getKeyPressed()
    if (key == "left" ):
        #print("left key is ressed")
        left_tor=0.1
        right_tor=-0.1
    elif (key=="right"):
        #print("right key is ressed")
        left_tor=-0.1
        right_tor=0.1
        #print("down key is pressed")
    data=synthesizeData(p,robot)
    vel=balance.callback(data,key)
    print(vel,data[1])
    
    p.setJointMotorControl2(robot, left_joint , p.TORQUE_CONTROL, force = -(vel+left_tor))
    p.setJointMotorControl2(robot, right_joint , p.TORQUE_CONTROL, force = vel+right_tor)
    p.stepSimulation()
    time.sleep(0.01)



# # Reference
# [1] Python Control Package , [link](https://github.com/markwmuller/controlpy)
# [2] Control System Tutorials , [link](http://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=SystemModeling)
# https://automaticaddison.com/linear-quadratic-regulator-lqr-with-python-code-example/

----------------------------------------------------------------------------------------------------------------
[0.] [ 0. -0.  0.]
----------------------------------------------------------------------------------------------------------------
[4.92570801e-40] [ 1.44560290e-20 -4.89789313e-43  6.77626358e-23]
----------------------------------------------------------------------------------------------------------------
[-1.93841653e-18] [4.91504985e-20 1.92747053e-21 2.33404634e-22]
----------------------------------------------------------------------------------------------------------------
[-1.79736846e-19] [7.99900269e-20 1.92747053e-21 3.91517451e-22]
----------------------------------------------------------------------------------------------------------------
[-4.05656991e-18] [1.15648232e-19 5.78241159e-21 5.72217813e-22]
----------------------------------------------------------------------------------------------------------------
[-6.35446014e-18] [1.59016319e-19 1.15648

----------------------------------------------------------------------------------------------------------------
[-0.05240611] [3.42331288e-06 3.76475546e-04 1.38938554e-03]
----------------------------------------------------------------------------------------------------------------
[-0.05042207] [7.06772394e-06 3.91704741e-04 1.43578637e-03]
----------------------------------------------------------------------------------------------------------------
[-0.05095311] [7.64776840e-06 4.06049872e-04 1.48269822e-03]
----------------------------------------------------------------------------------------------------------------
[-0.05231868] [8.25311679e-06 4.20422733e-04 1.52924480e-03]
----------------------------------------------------------------------------------------------------------------
[-0.05070195] [8.98029088e-06 4.31855294e-04 1.57557678e-03]
----------------------------------------------------------------------------------------------------------------
[-0.0481428] [1.0

----------------------------------------------------------------------------------------------------------------
[-0.04177247] [5.09013618e-05 3.93346955e-04 4.54750206e-03]
----------------------------------------------------------------------------------------------------------------
[-0.04021677] [5.46105652e-05 3.96864106e-04 4.64586865e-03]
----------------------------------------------------------------------------------------------------------------
[-0.03668276] [5.92535005e-05 3.96541081e-04 4.74984048e-03]
----------------------------------------------------------------------------------------------------------------
[-0.03904048] [5.73479147e-05 3.98592413e-04 4.85192244e-03]
----------------------------------------------------------------------------------------------------------------
[-0.0378226] [5.08608903e-05 3.99242531e-04 4.93915208e-03]
----------------------------------------------------------------------------------------------------------------
[-0.03629223] [4.4

----------------------------------------------------------------------------------------------------------------
[-0.03326661] [0.00010116 0.00040046 0.00780532]
----------------------------------------------------------------------------------------------------------------
[-0.03316796] [0.00012464 0.00039631 0.00789343]
----------------------------------------------------------------------------------------------------------------
[-0.04076591] [0.00011764 0.0004001  0.0079783 ]
----------------------------------------------------------------------------------------------------------------
[-0.03714024] [0.00011071 0.00039993 0.00806032]
----------------------------------------------------------------------------------------------------------------
[-0.03957079] [9.03438298e-05 4.02196189e-04 8.12909018e-03]
----------------------------------------------------------------------------------------------------------------
[-0.04166648] [8.50570659e-05 4.06334330e-04 8.19726529e-03]
----

----------------------------------------------------------------------------------------------------------------
[-0.03393328] [0.00014517 0.00041191 0.01279446]
----------------------------------------------------------------------------------------------------------------
[-0.04641115] [0.00013442 0.00041986 0.01291818]
----------------------------------------------------------------------------------------------------------------
[-0.04651612] [0.00014244 0.00042718 0.01304456]
----------------------------------------------------------------------------------------------------------------
[-0.03928643] [0.00015294 0.00042664 0.01317351]
----------------------------------------------------------------------------------------------------------------
[-0.04182695] [0.00012849 0.00042867 0.01329087]
----------------------------------------------------------------------------------------------------------------
[-0.0389183] [0.00011939 0.00042762 0.01340595]
-----------------------------

...

----------------------------------------------------------------------------------------------------------------
[-0.00903816] [-1.83285894e-04  2.70295160e-04  2.74439623e+00]
----------------------------------------------------------------------------------------------------------------
[0.02642419] [-1.20615131e-04  2.18957457e-04  2.73871564e+00]
----------------------------------------------------------------------------------------------------------------
[-0.09308308] [-6.65326193e-05  2.91212395e-04  2.73299297e+00]
----------------------------------------------------------------------------------------------------------------
[0.00272641] [-4.83405623e-05  2.61499132e-04  2.72723664e+00]
----------------------------------------------------------------------------------------------------------------
[-0.04273027] [-1.52383507e-04  2.79740986e-04  2.72166124e+00]
----------------------------------------------------------------------------------------------------------------
[-0.

----------------------------------------------------------------------------------------------------------------
[-0.18338608] [2.03342581e-05 3.31246731e-04 2.41698635e+00]
----------------------------------------------------------------------------------------------------------------
[0.06786463] [8.28262797e-05 2.33050953e-04 2.40967204e+00]
----------------------------------------------------------------------------------------------------------------
[-0.15409094] [4.68949815e-05 3.64662448e-04 2.40228222e+00]
----------------------------------------------------------------------------------------------------------------
[0.13281308] [8.11577933e-05 1.98786557e-04 2.39486262e+00]
----------------------------------------------------------------------------------------------------------------
[-0.08718236] [3.32946154e-05 2.67044413e-04 2.38755425e+00]
----------------------------------------------------------------------------------------------------------------
[-0.05289439] [-7.9

----------------------------------------------------------------------------------------------------------------
[-0.06808863] [-7.43245460e-05  2.98011046e-04  1.96191014e+00]
----------------------------------------------------------------------------------------------------------------
[-0.1277272] [-1.53340508e-06  3.97384344e-04  1.95254078e+00]
----------------------------------------------------------------------------------------------------------------
[-0.16907631] [-7.47424384e-05  5.28659012e-04  1.94299300e+00]
----------------------------------------------------------------------------------------------------------------
[-0.08505208] [-4.39491969e-06  5.64211678e-04  1.93337385e+00]
----------------------------------------------------------------------------------------------------------------
[0.00162985] [-4.45847834e-05  5.10275331e-04  1.92376338e+00]
----------------------------------------------------------------------------------------------------------------
[-0.

----------------------------------------------------------------------------------------------------------------
[-0.03475067] [9.15157694e-05 2.98942329e-04 1.44910904e+00]
----------------------------------------------------------------------------------------------------------------
[-0.07286409] [7.11458424e-05 3.43675975e-04 1.43994676e+00]
----------------------------------------------------------------------------------------------------------------
[-0.0100943] [1.54911594e-04 3.21846423e-04 1.43083417e+00]
----------------------------------------------------------------------------------------------------------------
[0.00065429] [1.52928624e-04 2.91353088e-04 1.42177047e+00]
----------------------------------------------------------------------------------------------------------------
[-0.04347261] [1.27934641e-04 3.07564923e-04 1.41269590e+00]
----------------------------------------------------------------------------------------------------------------
[0.12301981] [2.176

----------------------------------------------------------------------------------------------------------------
[-0.08852247] [-3.06869470e-05  3.63344592e-04  9.55113102e-01]
----------------------------------------------------------------------------------------------------------------
[-0.21729963] [3.61979100e-05 5.45726564e-04 9.46530746e-01]
----------------------------------------------------------------------------------------------------------------
[0.18892638] [8.54711140e-05 3.07265325e-04 9.37937503e-01]
----------------------------------------------------------------------------------------------------------------
[-0.14206774] [-3.43559777e-05  4.20040100e-04  9.29536069e-01]
----------------------------------------------------------------------------------------------------------------
[-0.02254947] [6.73777916e-06 4.03514630e-04 9.20820285e-01]
----------------------------------------------------------------------------------------------------------------
[-0.03087342